Ok so this here sheet is going to be the full image subtraction routine for SN photometry. Note that it is being written to be applied to a single filter at a time, for simplicity!

Note that following the original script from GROWTH 2020, this script will remove the processed directories when you run it, so careful!

This version does not require any alginment beforehand, but should be plate-solved (not entirely sure AIJ routine is good enough, might need to write another to handle that - there is one in the rotation sheet). **Without the plate solve, this script will throw you strange errors!**

**The source directory should contain just the source images**, and the configuration directory has a bunch of configuration files that this script needs. The data directory will be created by this code as the sources images + conifguration files, and the processed directory will be....the processed directory! This is awkward, but can't really figure out how to make it work otherwise...the out directory contains some random files that are produced in the process, as well as the final difference images.

Note you will have to move the processed files out of the processed/out/ directory into long-term storage of some kind.

Revision History:
<1.2: ?!?!?!

1.2: Does everything I hope?
    koy: Did a new WCS using the worksheet to fix alignment problems. **This worked!!**
    

20240813: Doing AT2023sse. This basically failed, you really can't see the source at all. I did modify the pfsex_config file to shrink the size of the PSF, and that still did not work. The subtracted images have multiple problems, including offsets, so perhaps I should try my plate solve method before giving up? Nope, better WCS didn't help :-/ Added a filter on the catalogs that calculate the normalization, to kill all flags over 4. Then give up!!! The filtered worked, but did not impact the resulting subtraction, so we are giving up!

CORRECTION: I've convinced myself that it's the size of the windowed region that borking things up, so just shrinking it until there are no saturated sources in it at all. That's just subtraction, so it shouldn't impact things toooooo much? Yup, it ran, and "the problem" was fixed, ugh.

1.3 Got a whole bunch more options now to deal with see - make sure the defaults are set below before you run the thing!

20240909: Doing AT2023umf. Making images smaller...

20240912: Doing AT2023pnt. AIJ WCS didn't really kill all the galaxy signal, so trying my Plate Solve routine. Galaxy is still present in the subtraction. Shrinking, debugging...There is a psf region hitting into the subtraction region, so shrinking the subtraction region. Might have fixed some, but not many. Remove outliers in flux ratio...removing background masking...reverting, that was probably slightly worse. Trying catalog filter before normalization. No help, but negative flux ratios!? Fixed my filter, STILL not subtracting the galaxy! Shrink window more? Ok, "last" thing to try is do the background subtraction before resampling...nope! Masking saturated starts from the beginning? (another possible solution - close saturated stars to target are getting PSF smeared in to the target! Ok, finally I have fixed the PSF problem by masking the saturated stars...I should implement that for all of the images, at the very beginning!

20241007: Doing AT2023yfh, with the new saturated star mask, right before PSF subtract (so background subtract is unaltered). Still looks pretty bad! turn up level, so more stars are masked? Well, it actually looks like the background subtraction is not working that well - lots of excess still there. Remove masking? Yes, that seemed to do a better job of removing the large scale gradients. Filter out the bright images left after subtraction...no help, try the outlier removal. Also no real help. Back to beginning, make entire image smaller? Ahh ok that actually kinda looks right! At least the source is remaining after removing....something! For B, subtraction somewhat less good, see if I can help. Saturation subtraction does not look right! Removing that and sticking with it.

20250103: Redoing SN2023cyx.

# Dependencies

There are a number of external codes that this process uses. I've managed to get them up and running on Pop_OS, but not Slackware, so these instructions are not a guarantee!

(there are other things that need to be install via pip, like photutils,image_registration,astroalign....)

SWarp https://www.astromatic.net/software, in apt but that version doesn't seem to do anything...trying from source. That lead me down the autogen rabbithole. Install autoconf,libtool via apt, then ./autogen.sh. Now configure and make.

SExtractor https://www.astromatic.net/software (this appears to be in apt...yeah it's a name change, it's source-extractor now!)

PSFex https://www.astromatic.net/software (in apt!)

ds9 http://ds9.si.edu/site/Home.html (seems to just be a binary, so putting it in /usr/local/bin!) (this is not actually used in this sheet, just including it for full information...it pops out the graphics in a new window and allows for interactions, so might be helpful later)

# "Instructions"

Just to be very clear, things that need to be done to make this sheet workable. Do this stuff FIRST - if you, for example, copy the files in after one of the cells that tried to copy them into the right place, you will fail!

1. Plate solve stacked images - AIJ first.
2. Copy images for a single filter into the "source" directory.
3. Change the source position in the cell below.
4. Change everything back to default before running for the first time!

In [1]:
# some parameters first.

# Images will be resampled to this size (might have x,y switched!)
# Ok, these seem not to impact the resampled files, which are the only thing we use.

# images generally need to be cropped down below their smallest dimension. Look for black edges and bad
# PSFs to be signs that this needs to be smaller.
SWARPSIZEX=400 #Default 1800
SWARPSIZEY=400 #Deault 900

# bg changes sometimes needed for smaller images
bg_size=30 # for smoothing the background estimator (default 50)
filter_size=3 # size for median filter of background estimator (default 3)
rad=10 #pixels for source subtract in the background, default 10 (tests with AIJ indicate 6 is source size...)

# source position (needed for the windowed subtraction scheme)
RA=206.275465063
Dec=3.80370200338
window=30 # size of windowed region (50 default) Be very careful, we need like 10-15 pixels of RADIUS for the photometry!

# Mask sources in background estimation
bg_mask_flag=1 # 1 for masked, 0 for unmasked!

# Now a flag for masking out saturated sources *completely* in the image. This was originally done to fix
# pnt, but it seems to be a fix for the bad PSF problem in previous images. It should maybe be the default!
# Note there is also a saturation level which can be set in the that block, lower down.
sat_mask_flag=1 # default 0
pixmask=1 # connected pixels needed to mask, so MORE means LESS masking
SAT_FRAC=0.80 # Above this saturation fraction, mask the sources

# We wrote a routine to filter the catalogs before normalization to remove flagged sources (generally saturated or
# other bad things) but that did not seem to improve the situation here (sse), so we are giving up! Change this flag
#if you like, but no evidence that it actually helps!
filter_cat_flag=1 # default 0
# in the outlier elimination, also - reduced the spread over the normalization, but didn't actually improve
# the resulting images! arg! This gives a 3 sigma filter on the flux ratios.
eliminate_outliers_flag=1 # default 0
show_images_flag=1 # default 0, just shows more information during the subtraction step.
extract_thresh=5 # Threshhold for detection above sigma - default 5

# This flag is going to implement the background subtraction routine BEFORE the resampling step.
# This is not the original behavior of the GROWTH workshop, but we are trying to deal with pnt,
# and the subtraction seems to be very inconsistent
bg_subtract_pre_resamp_flag=0 # default 0 Didn't help the first time I tried it!

#sometimes we need to make this smaller, when the noise is such that the signal in the PSF doesn't dominant
# comes as a normalization warning.
size_of_psf=31 #default 59 - must be odd!



First import relevant packages:

In [2]:
import os  #Call commands from outside Python
import numpy as np

# Running external programs
import subprocess
import shutil

from astropy.coordinates import SkyCoord # For the catalog analysis during normalization
import astropy.units as u # for coordinate units

from astropy.io import fits #FITS files handling
from astropy.io import ascii  #Read/write ascii files
from astropy.wcs import WCS # for the cropping routine


# Background subtraction
import photutils
#from photutils.detection import DAOStarFinder
from photutils.segmentation import detect_threshold, detect_sources
#from photutils import Background2D, MedianBackground # old version
from photutils.background import Background2D, MedianBackground 
from astropy.stats import sigma_clipped_stats, SigmaClip # statistics
from photutils.utils import circular_footprint

# Image registration and shifting
from image_registration import chi2_shift
from image_registration.fft_tools import shift
import scipy
from scipy import ndimage, misc
from astropy.table import Table # for catalog manipulation

# Useful to smooth the images with a Gaussian kernel before the subtraction
from scipy.signal import convolve as scipy_convolve
from astropy.convolution import convolve, convolve_fft # for saturation star masks during convolution.

# For rotation
import astroalign as aa

# Plot
import matplotlib.pyplot as plt

# Math help
import math

# Aperture photometry at the end
from photutils.aperture import CircularAperture, ApertureStats

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Set directory structure
cwd = os.getcwd()

print("You start from the directory:", cwd)

source_dir = os.path.join(cwd, 'source') # All the stacked, rotated files should be in here
data_dir = os.path.join(cwd, 'data') # empty, but will get rewritten.
if os.path.isdir(data_dir): # checks and removes if it exists. 
    shutil.rmtree(data_dir)
os.mkdir(data_dir)
proc_dir = os.path.join(cwd, 'processed') # all processes fits files.
out_dir = os.path.join(proc_dir, 'out') # output files are here
if os.path.isdir(proc_dir): # checks and removes if it exists. 
    shutil.rmtree(proc_dir)
os.mkdir(proc_dir)
config_dir = os.path.join(cwd, 'config') # various configuration files, should exist!
if os.path.exists(config_dir)==False:
    print("Error, config directory does not exist!")
    raise

for f in os.listdir(source_dir): # copy sources into data directory.
    shutil.copy2(os.path.join(source_dir, f), os.path.join(data_dir,f))
for f in os.listdir(config_dir): # copy configuration into data directory.
    shutil.copy2(os.path.join(config_dir, f), data_dir)
for f in os.listdir(data_dir): # copy data into processed and change to it.
    shutil.copy2(os.path.join(data_dir, f), proc_dir)

os.chdir(proc_dir)
print("You are working in the image_subtraction/processed/ directory: ")
print("Full path:", proc_dir)

You start from the directory: /media/cduston/MCARG/AnalyzedSources/SN2023cyx/image_subtraction/OIS_test
Error, config directory does not exist!


RuntimeError: No active exception to reraise

In [ ]:
rot_name_array=[]
for f in os.listdir(source_dir): # create list of raw images
    rot_name_array.append(f)
    
rot_name_array.sort() # sorted by name so that discovery is first.
rot_name_array

## Subtraction Routine

This part comes from the GROWTH 2020 school, with my own modifications and dealing with some recent changes to the code. We'll start with a dependency check:

(for now skipping all reference to ds9, since that doesn't actually seem that useful atm.)

---

>This box is for FAILED Slackware installation!

>installing these things was kind of a nightmare...these instructions are for Slackware...

>Swarp: https://www.astromatic.net/software download source. run autogen? ./configure, make, su, make install...

>SExtractor https://www.astromatic.net/software source....will need ATLAS and FFTW. trying autogen. ATLAS not in 15.0 yet! Guess I'll switch to System76 and try old Slackbuilds. Ugh ATLAS does not seem to build on Slack 15.0 (email thread). Trying the intel MKL approach...following wget instructions from https://www.intel.com/content/www/us/en/developer/tools/oneapi/base-toolkit-download.html?operatingsystem=linux&distributions=offline. Compatibility warnings, proceeding...isntall location /opt/intel/oneapi. Failing lots of prereqs. All gui, so just pushing forward. Skip eclipse. Need g++, but I actually have that installed, it just can't find it. continuing...says done. trying to configure with --enable-mkl, but cannot find c compilter. 

>ATLAS alone does not seem to build either on Slackware.....bust!

---

In [ ]:
def test_dependency(dep, alternate_name=None):
    """
    Test external dependency by trying to run it as a subprocess
    """
    try:
        subprocess.check_output(dep, stderr=subprocess.PIPE, shell=True)
        print("%s is installed properly as %s. OK" % (dep, dep))
        return 1
    except subprocess.CalledProcessError:
        try:
            subprocess.check_output(alternate_name, stderr=subprocess.PIPE, shell=True)
            print("%s is installed properly as %s. OK" % (dep, alternate_name))
            return 1
        except subprocess.CalledProcessError:
            print("===%s/%s IS NOT YET INSTALLED PROPERLY===" % (dep, alternate_name))
            return 0

#dependencies = [('sextractor', 'sex'), ('SWarp', 'swarp'), ('psfex', 'PSFEx'), ('ds9', 'DS9')]
#dependencies = [('sextractor', 'source-extractor'), ('SWarp', 'swarp'), ('psfex', 'PSFEx'), ('ds9', 'DS9')]
dependencies = [('sextractor', 'source-extractor'), ('SWarp', 'swarp'), ('psfex', 'PSFEx')]
i = 0
for dep_name1, dep_name2 in dependencies:
    i += test_dependency(dep_name1, dep_name2)
print("%i out of %i external dependencies installed properly.\n" % (i, len(dependencies)))
if i != len(dependencies):
    print("Please correctly install these programs before continuing by following the instructions in README.md.")
else:
    print("You are ready to continue.")   

## Align the images

Use the AstrOmatic Swarp package to align the images.  Swarp relies on the astrometric information of the image (in other words, on the sky coordinates), therefore both the science and reference images must be astrometrically calibrated (for example, using the AstrOmatic SCAMP package).  In this module we assume that the input images are already calibrated.

We are going to pass the entire list to swarp at once, methinks. Create the swarp filelist dynamically, to reduce the kinds of errors we will make in the future!

In [ ]:
# rot_image_array=[]
swarp_filelist_name=config_dir+"/"+'swarp_filelist.txt'
swarp_filelist = open(swarp_filelist_name, 'w')
for f in rot_name_array: 
    #rot_image_array.append(f)
    print(f)
    swarp_filelist.write(str(f) + "\n")
swarp_filelist.close()


Note in this next command is what crops the images...

In [ ]:
# Swarp command
# had to change capitalization of SWarp!
try:
    #command = "swarp %s %s -c %s -SUBTRACT_BACK N -RESAMPLE Y -RESAMPLE_DIR . -COMBINE N -IMAGE_SIZE 1800,900" % (sci_image_name, ref_image_name, os.path.join(config_dir, 'config.swarp'))
    if bg_subtract_pre_resamp_flag==0:
        command = "swarp @%s -c %s -SUBTRACT_BACK N -RESAMPLE Y -RESAMPLE_DIR . -COMBINE N -IMAGE_SIZE %s,%s" % (swarp_filelist_name, os.path.join(config_dir, 'config.swarp'),SWARPSIZEX,SWARPSIZEY)
    elif bg_subtract_pre_resamp_flag==1:
        print("Subtracting Background Before resampling...\n")
        command = "swarp @%s -c %s -SUBTRACT_BACK Y -RESAMPLE Y -RESAMPLE_DIR . -COMBINE N -IMAGE_SIZE %s,%s" % (swarp_filelist_name, os.path.join(config_dir, 'config.swarp'),SWARPSIZEX,SWARPSIZEY)
    print('Executing command: %s' % command)
    rval = subprocess.run(command.split(), check=True)
    print('Success!')
except subprocess.CalledProcessError as err:
    print('Could not run SWarp with exit error %s'%err)

# Gotta fix the names of the aligned images
swarp_filelist = open(swarp_filelist_name, 'r')
align_image_array=[]
for line in swarp_filelist:
    #print(line)
    align_image_array.append(line.strip().replace(".fits", ".resamp.fits").replace('data','processed'))
#sci_image_aligned_name = sci_image_name.replace(".fits", ".resamp.fits").replace('data','processed')
#ref_image_aligned_name = ref_image_name.replace(".fits", ".resamp.fits").replace('data','processed')
swarp_filelist.close()

In [ ]:
align_image_array


Can use AstroImageJ to verify this basically looks right. - putting in a image check step here, since the next step sometimes gets borked.

In [ ]:
for i in range(len(align_image_array)):
    image = fits.open(align_image_array[i])
    mean, median, std = sigma_clipped_stats(image[0].data)
    plt.figure(figsize=(8,8))

    # set the scale of the image based on its statistics
    # Name looks totally wrong!
    plt.imshow(image[0].data, vmin=median-2*std, vmax=median+2*std)
    plt.colorbar(shrink = 0.4)
    plt.title('Resampled image: '+align_image_array[i])
    plt.show()



# Background Subtraction
- Mask sources in images
- Use 3 sigma clipping to filter data and accurately measure the backgorund
- Then split image into 300x300 pixel boxes and apply 2x2 median filter

That was the original. We are going to (for the first image)

1) look at the scalar background (for no good reason!)
2) look at the 2D background with no mask
3) look at the 2D background a mask

And then assume (3) is best and go ahead with subtracting them all!

Note that the final background subtraction is skipped if the flag indicating that it should already have
happened during the resampling step is set to 1!

In [ ]:
# Let's loop through the images and check the essential statistics on each background.

sigma_clip = SigmaClip(sigma=3.0, maxiters=20) # parameters for estimation
footprint = circular_footprint(radius=rad) #size of source subtraction?
print("Mean, Median, and Standard Deviation for the background in each image\n\n")
for i in range(len(align_image_array)):
    image = fits.open(align_image_array[i])
    # Eventually do this for each image in the array.
    #image = sci_image_aligned # remove for multi image!
    #image = fits.open(align_image_array[0])
    hdr = image[0].header #save fits header
    threshold = detect_threshold(image[0].data, nsigma=2.0, sigma_clip=sigma_clip)
    segment_img = detect_sources(image[0].data, threshold, npixels=10)
    
    # This next line fails sometimes, so let's put in some debugging
    #plt.imshow(segment_img, origin='lower', cmap=segment_img.cmap,interpolation='nearest')
    #plt.colorbar()
    #plt.show()
    
    mask = segment_img.make_source_mask(footprint=footprint)
    mean, median, std = sigma_clipped_stats(image[0].data, sigma=3.0, mask=mask)
    print((mean, median, std))  

I guess just look for outliers....

cyx, (first try) some of the images are super duper noisy and quite a bit brighter than the others! (second try) Same! Plus tons of hot pixels now?

koy: (R) Yeah, second one is noise but not crazy so. (U) noise is quite small, even though the signal looks pretty bad.

sse: Some more, some less, seems fine.

2D background estimator (50x50 box, 3x3 median filter). First image first.

In [ ]:
bkg_estimator = MedianBackground()
sigma_clip = SigmaClip(sigma=3.0)
bkg = Background2D(image[0].data, (bg_size,bg_size), filter_size=(filter_size, filter_size),sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
plt.imshow(bkg.background, origin='lower', cmap='Greys_r',interpolation='nearest')
plt.colorbar()

But this is unmasked, and it does look like the stars are impacting the background. So let's try the same with a mask.

In [ ]:
bkg = Background2D(image[0].data, (bg_size,bg_size), filter_size=(filter_size, filter_size),sigma_clip=sigma_clip, bkg_estimator=bkg_estimator,mask=mask)
plt.imshow(bkg.background, origin='lower', cmap='Greys_r',interpolation='nearest')
plt.colorbar()

yyyeaahh I guess that's better, the background should probably be NOT bumpy at all. So use this and background subtract everything.

(Background goes negative here, I had real problems in AIJ with that in the past, so make a note of that!)

sse: (large image) Uhhhh background was totally flat, which seems wrong...it's not just the scale, the mask must actually be bonkers. Start by trying unmasked background. (small image) looks way more even, let's do it. still failing, so let's try unmasked? No, unmasked went negative!

So actually, sometimes the unmasked background here looks *better*, at least with this first image. So I'm going to set a flag to decide which background to use. This should probably be set to default to the masked, however!

In [ ]:
# Do this for each image in the array.
bkg_subtract_array=[]
for i in range(len(align_image_array)):
    image = fits.open(align_image_array[i])
    hdr = image[0].header #save fits header
    
    # Basic background set up.
    bkg_estimator = MedianBackground()
    sigma_clip = SigmaClip(sigma=3.0)
    threshold = detect_threshold(image[0].data, nsigma=2.0, sigma_clip=sigma_clip)
    
    # get source mask individually for each image (didn't have to do this on ctn!)
    if bg_mask_flag==1:
        # get source mask individually for each image (didn't have to do this on ctn!)
        print("Subtracting background with a mask...")
        segment_img = detect_sources(image[0].data, threshold, npixels=10)
        mask = segment_img.make_source_mask(footprint=footprint) # mask too big once?
        # sometimes need to evaluate the mask...
        #print(mask.sum())
        # measure and subtract the background
        # note this exclude_percentile is default 10, and more means more exclusions. From the Docs, "For best results, 
        # exclude_percentile should be kept as low as possible ", although increasing has fixed a few errors.
        bkg = Background2D(image[0].data, (bg_size,bg_size), filter_size=(filter_size,filter_size),\
                       sigma_clip=sigma_clip, bkg_estimator=bkg_estimator,mask=mask,exclude_percentile=10.0)
    if bg_mask_flag==0:
        # use unmasked background.
        print("Subtracting background without a mask...")
        bkg = Background2D(image[0].data, (bg_size,bg_size), filter_size=(filter_size, filter_size),sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)

    # write background file
    bkg_subtract_name=align_image_array[i].replace(".resamp.fits", "_bkgsub.fits")
    bkg_subtract_array.append(bkg_subtract_name)
    if bg_subtract_pre_resamp_flag==0: # do this background subtraction...
        image[0].data = image[0].data - bkg.background
    elif bg_subtract_pre_resamp_flag==1: # don't do it, it's already been done!
        print("Background subtraction already happened during resampling step!\n")
    hdu_image_sub = fits.PrimaryHDU(image[0].data,image[0].header) #added headers! Includes SAT level!
    hdu_image_sub.writeto(bkg_subtract_name,overwrite=True)
    
    print("Mean, Median, and Standard Deviation for the background\n\n")
    mean, median, std = sigma_clipped_stats(image[0].data, sigma=3.0, mask=mask)
    print((mean, median, std))  
    print("Writing background subtracted file",bkg_subtract_name,"\n===============================================")
    #plotting for evaluation
    fig, axs = plt.subplots(1, 2,figsize=(32,32)) # could add figsize command...
    plt.title(bkg_subtract_name)
    mean, median, std = sigma_clipped_stats(bkg.background)
    pcm=axs[0].imshow(bkg.background, vmin=median-2*std, vmax=median+2*std)
    plt.colorbar(pcm,ax=axs[0],shrink=0.4)
    mean, median, std = sigma_clipped_stats(image[0].data)
    pcm=axs[1].imshow(image[0].data, vmin=median-2*std, vmax=median+2*std)
    plt.colorbar(pcm,ax=axs[1],shrink=0.4)

For cyx, (first time) unmasked background was *way* better, much less gradient present after subtraction. (1.6) Also true, unmasked looks better.

koy: (R) Yeah second image has some kind of crazy arc in it. Leaving it for now. (U) Ok, these look *ok*, but not amazing. They look aligned, at least, but the target is totally invisible.


sse: Mostly seems ok, slightly noiser around edges. Progressing for now. 

These images look right in AIJ, but I did get an error that these images are now double precision, whereas AIJ expects single precision. Fine for now?

In [ ]:
bkg_subtract_array

## PSF matching

The atmosphere heavily affects the PSF of the images by determining the "seeing" conditions. The seeing for ground-based optical telescopes is usually measured as the FWHM of the imaging PSF.  Properties of the atmosphere can change very rapidly, so it is rare that science and reference images are characterized by the same seeing. Therefore their PSFs are usually different, which is a problem for image subtraction. 


### Generate the kernel for the convolution

The PSF of the science and reference images can be matched in several different ways.  Here we start by performing a first source extraction on the reference image (which we assume to be the last in the list).  We can use the catalogs of sources that we obtain for two main purposes: <br />
1. Measure the PSF of the science reference frame, using PSFex or photutils
2. Obtain instruments magnitudes that will be the basis for the zero-point calibration (see Photometry module).  


In [ ]:
# Block for masking out the saturated stars.

if sat_mask_flag==1:
    for i in range(len(bkg_subtract_array)):
        # grab the saturation level ...
        file_name=os.path.join(proc_dir, bkg_subtract_array[i])
        # get saturation level
        image = fits.open(file_name) # only need open to read the header
        SAT=image[0].header['SATURATE']
        print("Saturation level",SAT)
        # Check histogram?
        #mean, median, std = sigma_clipped_stats(image[0].data)
        #plt.imshow(image[0].data,vmin=median-2*std,vmax=median+2*std)
        #plt.show()
        #hist_data=(image[0].data).flatten()
        #print(hist_data.shape,(image[0].data).shape)
        #plt.hist(hist_data, bins=1000,range=(0,40000))
        #plt.show()

        
        # No, this block needs to mask out a CIRCLE around the saturated source, otherwise we get those missing
        # black spots in the PSF...

        # set the threshhold to like 80% of that level....
        limit=SAT_FRAC*SAT
        # use segment images to mask the sources which have any pixels over that 80% level...
        # 1 connected pixels greater than the limit. Not that a larger value here means LESS masking.
        # npixels=4 worked before, but found nothing in B filter...
        segment_map = detect_sources(image[0].data, limit, npixels=pixmask)
        #print(segment_map)
        #plt.imshow(segment_map, cmap=segment_map.cmap,interpolation='nearest')
        #plt.show()
        # ...
        # profit?
        if (segment_map == None): # if there are no saturated sources, skip this!
            masked_image=image[0].data
        else:
            mask_bool=segment_map.make_source_mask()
            masked_image=np.where(mask_bool==False,image[0].data,np.NaN)
        #mean, median, std = sigma_clipped_stats(masked_image)
        #plt.imshow(masked_image,vmin=median-2*std,vmax=median+2*std)

        fig, axs = plt.subplots(1, 2,figsize=(16,16)) # could add figsize command...
        plt.title(file_name)
        mean, median, std = sigma_clipped_stats(mask_bool)
        pcm=axs[0].imshow(mask_bool, vmin=median-2*std, vmax=median+2*std)
        plt.colorbar(pcm,ax=axs[0],shrink=0.4)
        mean, median, std = sigma_clipped_stats(masked_image)
        pcm=axs[1].imshow(masked_image, vmin=median-2*std, vmax=median+2*std)
        plt.colorbar(pcm,ax=axs[1],shrink=0.4)

        # now write the masked image to the original image location, so the names don't have to get messed around with.
        hdu_image_sub = fits.PrimaryHDU(masked_image,image[0].header) #added headers! Includes SAT level!
        hdu_image_sub.writeto(file_name,overwrite=True) # overwrite the files so we can continue without change.

else:
    print("Vanilla routine, saturated stars NOT masked")

In [ ]:
# use REFERENCE image to extract sources.
# overwriting the catalog, but I guess that doesn't matter much.

#ref_image_bkgsub_name = ref_bkg_subtract_name # SWTICH when you go back to multi image!
ref_image_bkgsub_name = os.path.join(proc_dir, bkg_subtract_array[len(bkg_subtract_array)-1])

if os.path.exists('prepsfex.cat'): #Remove possible temporary files
    os.remove("prepsfex.cat") 

# get saturation level
ref_image_bkgsub = fits.open(ref_image_bkgsub_name) # only need open to read the header
SAT=ref_image_bkgsub[0].header['SATURATE']
print("Saturation level",SAT)
    
try:
    #command = "sextractor %s -c %s -CATALOG_NAME %s -MAG_ZEROPOINT 25.0" % (sci_image_aligned_name, os.path.join(data_dir, 'prepsfex.sex'), os.path.join(proc_dir, 'prepsfex.cat'))
    #command = "source-extractor %s -c %s -CATALOG_NAME %s -MAG_ZEROPOINT 25.0" % (sci_image_aligned_name, os.path.join(data_dir, 'prepsfex.sex'), os.path.join(proc_dir, 'prepsfex.cat'))
    command = "source-extractor %s -c %s -CATALOG_NAME %s -MAG_ZEROPOINT 25.0 -SATUR_LEVEL %s" % (ref_image_bkgsub_name, \
        os.path.join(data_dir,'prepsfex.sex'), os.path.join(proc_dir, 'prepsfex.cat'),SAT)
    print('Executing command: %s\n' % command)
    rval = subprocess.run(command.split(), check=True)
    print('Success!')
except subprocess.CalledProcessError as err:
    print('Could not run SExtractor with exit error %s'%err)

Saturation filter pre-PSF, didn't do much so generally not used.

In [ ]:
# look for saturation (ANY 4, even in another number, indicates a saturated pixel):
# But note from the docs:  https://sextractor.readthedocs.io/en/latest/Flagging.html these flags are
# "in order of increasing concern", so maybe the right is to filter out 4 AND HIGHER?
# Need the lines above for this!
'''
filter_sat=1 # 0 to NOT filter out saturation flags, 1 to YES filter for saturation flags.
j=0
for i in reversed(range(len(cat_data['FLAGS']))): #loop backwards so we can remove rows
    if cat_data['FLAGS'][i]==4:
        if filter_sat==1:
            cat_data.remove_row(i)
        j=j+1
    
j
'''

koy: (R) 52 sources from 727 have the saturation flag, but note this just removes the 4 flags ONLY - if they have multiple flags they are untouched. The saturation flag filtering did NOT help the alignment issue later. (U) no saturated flags?

sse: 153 of 1533 saturated flags on default settings.

From "Sextractor for dummies": https://arxiv.org/pdf/astro-ph/0512139

"SATUR LEVEL is the limit for SE to start extrapolating to get the photometry. However as
soon as you hit something as saturated as that you might want to see if there is another way to
determine the flux from that object."

So yes - it's just flagged. Good for catalog, but not good for any calculations!

PSFex: It seems to be ignoring ANY flagged sources, at least by default. I am going to change the parameter file, 

SAMPLE_MINSN       3            # Minimum S/N for a source to be used

To 20 (program default), and check the effect! No improvement! 50? Nothing! That's not the problem.

I guess take a look at the catalog...

In [ ]:
'''
fig = plt.figure(figsize=(8,8))
#ax = fig.add_subplot(111)
plt.scatter(cat_data['X_IMAGE'], cat_data['Y_IMAGE'], facecolors='none', edgecolors='r', linewidths=0.5)
#image = fits.open(align_image_array[i])
mean, median, std = sigma_clipped_stats(ref_image_bkgsub[0].data)
plt.imshow(ref_image_bkgsub[0].data, vmin=median-2*std, vmax=median+2*std)
'''

I think I would like a way to remove the saturated sources, since those might be causing the alignment problems I'm having with koy. 

Now I have removed them, but clearly not all of them.

Now we use another software part of the AstrOmatic suite, PSFex, to **measure the PSF of the reference image**. PSFex estimates the PSF based on the information present in the catalog generated with SExtractor.  Then, let's plot the PSF model obtained with PSFex

In [ ]:
# Run PSFex to compute PSF, read and display the final model; needs to output to "out" dir.
if not os.path.isdir('out'): os.mkdir('out')

try:
    command = "psfex prepsfex.cat -c psfex_conf.psfex -PSF_SIZE=%s, %s -VERBOSE_TYPE=FULL" % (size_of_psf,size_of_psf)
    
    #command = "psfex %s -c %s" % (source_extract_cat_name,psfex_config_name)
    print('Executing command: %s\n' % command)
    rval = subprocess.run(command.split(), check=True)
    print('Success!')
except subprocess.CalledProcessError as err:
    print('Could not run psfex with exit error %s'%err)

psf_ref_image_name = os.path.join(out_dir,'proto_prepsfex.fits') # hard coded name from psfex?
print(psf_ref_image_name)
psf_ref_image = fits.open(psf_ref_image_name)

mean, median, std = sigma_clipped_stats(psf_ref_image[0].data[0])
plt.figure(figsize=(8,8))
plt.imshow(psf_ref_image[0].data[0], vmin=median-2*std, vmax=median+2*std)
plt.colorbar(shrink = 0.4)

cyx: (1.?) Another pretty bad PSF! Ok, but the median filtering helped! (1.6) hole in the middle? Fixed with a median on the reference, but still looks quite bad. Jesus finally, just screwing up WCS, B instead of I, etc.

koy: Seems reasonable.

### Convolve the science images with the PSF of the reference image

Now that the kernel is generated, let's convolve the science images with the PSF of the reference, to create our array of convolved science images.

In [ ]:
# Convolve the kernel of the reference image PSF with each science frame
kernel_ref = psf_ref_image[0].data[0]

# for plotting
#fig, axes = plt.subplots(3, math.ceil(len(bkg_subtract_array)/3.))
#plt.figure(figsize=(4,4))

sci_conv_array=[]
for i in range(len(bkg_subtract_array)-1): # don't need to do the last one, that's reference
    sci_bkg_subtract = fits.open(bkg_subtract_array[i])
    if sat_mask_flag==1: # can't do "fft" method, I guess! 
        sci_conv=convolve(sci_bkg_subtract[0].data, kernel_ref)
        # scipy convolve copies all the NaNs into the convolution.
        #sci_conv = scipy_convolve(sci_bkg_subtract[0].data, kernel_ref, mode='same', method='direct')
    else:
        sci_conv = scipy_convolve(sci_bkg_subtract[0].data, kernel_ref, mode='same', method='fft') # convolve step
    hdu_sci_conv = fits.PrimaryHDU(sci_conv,sci_bkg_subtract[0].header) # really should be adding header keys!
    sci_conv_array.append(bkg_subtract_array[i].replace(".fits","_conv.fits"))
    sci_conv_name=os.path.join(proc_dir,sci_conv_array[i])
    
    # Write the images
    print(sci_conv_name)
    hdu_sci_conv.writeto(sci_conv_name,overwrite=True)

    #Plot up the convolved reference image
    mean, median, std = sigma_clipped_stats(hdu_sci_conv.data)
    plt.figure(figsize=(8,8))

    # set the scale of the image based on its statistics
    # Name looks totally wrong!
    plt.imshow(hdu_sci_conv.data, vmin=median-2*std, vmax=median+2*std)
    plt.colorbar(shrink = 0.4)
    plt.title('Convolved science image')
    plt.show()

This step was borked for a while, and displaced the PRE-convolution images. These always look bad, on what looks like the saturated stars. I think this makes sense - when you have saturation, the PSFs are "too wide", and the presumably the psf generator is getting rid of some saturated sources.

koy: (U) These are clearly "right", but the noise is pretty high and this might be why the resulting images don't look so good.

sse: Looks pretty good.

Next step is actually to create a corresponding refernce image for each science image, which is the reference convolved with the science. Then each pair should have the same PSF.

In [ ]:
sci_conv_array

### Convolve the reference image with the PSF of each science image


Same as above, but this time we generate a kernel with the properties of the **PSF of the science images**.  Then, we convolve the reference image with this kernel, once for each science image. This creates the array of convolved reference images, which are needed to subtract from the science frames.

I think what is happening here is the PSF data is stored in prepsfex.cat, as an output catalog from source-extractor, so we need to re-run the source extractor to get the PSF of the image in question. 

Kernel from *each* science image

Maybe what needs to go here is a way to CLEAN the catalogs, removing sources that are clearly saturated, so they are not used in the normalization routine?


During this step, I once got an error (on sse) not finding sources with appropriate FWHM...I fixed that by adding -DETECT_MINAREA 1 to the source-extractor command (default 5), or -DETECT_THRESH 3...the second one looked a bit better. Basically, the seeing was just *too good* for that night! That's this line in the 3rd box wwwaaayyy above:

extract_thresh=5 # Threshhold for detection above sigma - default 5

In [ ]:
ref_conv_array=[]
for i in range(len(bkg_subtract_array)-1): # don't need to do the last one, that's the reference
    sci_image_bkgsub_name = os.path.join(proc_dir, bkg_subtract_array[i])
    sci_image_bkgsub = fits.open(sci_image_bkgsub_name) # only need open to read the header
    print("=======================================================\n\nThis Image info:") # for debugging
    sci_image_bkgsub.info() # for debugging

    try:
        # prepsfex.sex is the config file
        # prepsfex.cat is a binary output file, gets rewriten each time.
        # let's try to put the saturation level into this command.
        # didn't fix anything!
        # BACK_SIZE also doesn't seem to help this problem.
        SAT=sci_image_bkgsub[0].header['SATURATE']
        print("Saturation level",SAT)
        command = "source-extractor %s -c %s -CATALOG_NAME %s -MAG_ZEROPOINT 25.0 -SATUR_LEVEL %s -DETECT_THRESH %s" % (sci_image_bkgsub_name, \
            os.path.join(data_dir,'prepsfex.sex'), os.path.join(proc_dir, 'prepsfex.cat'),SAT,extract_thresh)
        
        print('Executing command: %s\n' % command)
        rval = subprocess.run(command.split(), check=True)
        print('Success!')
    except subprocess.CalledProcessError as err:
        print('Could not run SExtractor with exit error %s'%err)
     
    # Run PSFex to compute PSF, read and display the final model; needs to output to "out" dir.
    if not os.path.isdir('out'): os.mkdir('out') # should be made already

    try:
        # prepsfex.cat is the binary file created in the last step.
        # psfex_conf.psfex is config file, used each time.
        
        # can I filter out the saturated sources here, so they are not used for finding the PSF?
        
        command = "psfex prepsfex.cat -c psfex_conf.psfex -PSF_SIZE=%s, %s -VERBOSE_TYPE=FULL" % (size_of_psf,size_of_psf)
        print('Executing command: %s\n' % command)
        rval = subprocess.run(command.split(), check=True)
        print('Success!')
    except subprocess.CalledProcessError as err:
        print('Could not run psfex with exit error %s'%err)

    psf_sci_image_name = os.path.join(out_dir,'proto_prepsfex.fits') # hard coded name
    #psf_sci_image_name = os.path.join(out_dir,bkg_subtract_array[i].replace('.fits','_protoPSF.fits')) # rewriting each time!
    #print(psf_sci_image_name)
    psf_sci_image = fits.open(psf_sci_image_name)
    # only the last image shows...
    mean, median, std = sigma_clipped_stats(psf_sci_image[0].data[0])
    plt.figure(figsize=(8,8))
    plt.imshow(psf_sci_image[0].data[0],   vmin=median-2*std, vmax=median+2*std)  
    plt.colorbar(shrink = 0.4)
    plt.title('Science Image PSF')
    plt.show()   
        
    # Convolve the reference image PSF with the Pof each science frame
    # Grab the kernel!
    kernel_sci = psf_sci_image[0].data[0]
    
    # careful naming scheme needed!
    ref_bkg_subtract = fits.open(bkg_subtract_array[len(bkg_subtract_array)-1]) # REFERENCE IS LAST IMAGE!
    print("Check: Reference image here is",bkg_subtract_array[len(bkg_subtract_array)-1])
    if sat_mask_flag==1: # can't do "fft" method, I guess!
        ref_conv=convolve(sci_bkg_subtract[0].data, kernel_sci)
        # scipy convolve copies all the NaNs into the convolution.
        #ref_conv = scipy_convolve(ref_bkg_subtract[0].data, kernel_sci, mode='same', method='direct') # convolve step
    else:
        ref_conv = scipy_convolve(ref_bkg_subtract[0].data, kernel_sci, mode='same', method='fft') # convolve step
    
    hdu_ref_conv = fits.PrimaryHDU(ref_conv,ref_bkg_subtract[0].header) # header keys don't super matter...but from the reference!
    
    # yikes, this has to be named CAREFULLY!
    ref_conv_array.append(bkg_subtract_array[i].replace(".fits","_refconv.fits"))
    ref_conv_name=os.path.join(proc_dir,ref_conv_array[i])
    
    # Create the images
    print(ref_conv_name)
    hdu_ref_conv.writeto(ref_conv_name,overwrite=True)

    #Plot up the convolved reference image
    mean, median, std = sigma_clipped_stats(hdu_ref_conv.data)
    plt.figure(figsize=(8,8))

    # include the catalog
    cat = fits.open("prepsfex.cat")
    cat_data = Table(cat[2].data)
    plt.scatter(cat_data['X_IMAGE'], cat_data['Y_IMAGE'], facecolors='none', edgecolors='r', linewidths=0.5)
    
    # set the scale of the image based on its statistics
    plt.imshow(ref_conv, vmin=median-2*std, vmax=median+2*std)
    plt.colorbar(shrink = 0.4)
    plt.title('Convolved reference image for this science frame:')
    plt.show()    

Checking in AIJ, the resulting images have nearly the same PSF! (checking a few example sources, and they even seem to have the same pixel location values, which is good for us!

bright stars do have halos, which might be screwing up the background calculation, or the PSF? If we are eventually going to just subtract a circle, this might not matter.


sse: (large image) the first image has a bonkers PSF, with a giant line through it! (smaller image) fixed!

In [ ]:
ref_conv_array

### Improving the alignment
Now that the science image is convolved with (an approximation of) the PSF of the reference image, 
and the reference image is convolved with the PSF of the science image, we can perform the image subtraction.

- Before the subtraction we use an fft method (chi_2_shift) to fine-tune the image alignment of the reference and science image. We'll do this in pairs, reference aligning with science. When we go do photometry with AIJ, this fine tuning doesn't matter so much.

## Normalization and Subtraction of the images

The science and reference images are usually obtained with different exposure times.  In addition, the reference image can be the stack of several images to increase the depth.  Finally, different CCDs of the same camera (or even different regions of the same CCD when multiple amplifiers are present) may have slightly different gain. <br >

The background subtraction should have removed the non-linear offsets between science and reference images.  We can therefore normalize the two images by computing the ratio of bright star fluxes in the two images. Once again, we use SExtractor to extract the flux and other quantities.

We should be able to do this pairwise as well, as the offsets between different science images should be taken care of with our differential photometry. So we'll maybe normalize the reference(s) to the science?

Adding a catalog filter here, to get rid of things with flags over 4.

Have several other types of filters (and debugging info) down there, but no evidence that any of them actually helps anything!

In [ ]:
# cleaning - do this before matching, I think.
# Catalog expected is catalog filename from sextractor.
# Nah, I think this has to be done as ASCII to match the rest of the code!
# Maybe read it in twice - once to get the header (as "basic"), and once to get the data (as "sextractor").
# For the flags we need the sextractor format, while for the data we need ascii.
# so we will probably have to write as ascii....or correlate the rows between the two?
def clean_cat(catalog):
    cat_sci_sex = ascii.read(catalog,format='sextractor')
    cat_sci_basic = ascii.read(catalog,format='no_header') 
    print("Initial catalog length: ",len(cat_sci_sex),",",len(cat_sci_basic)," (for debug, should be the same!)")
    j=0
    for i in reversed(range(len(cat_sci_sex))): #loop backwards in sextractor catalog so we can remove rows
        if cat_sci_sex['FLAGS'][i]>=4:
            cat_sci_basic.remove_row(i) # remove the row in the basic catalog
        if cat_sci_sex['FLUX_AUTO'][i]<=0: # remove if flux drops below zero
            cat_sci_basic.remove_row(i) # remove the row in the basic catalog
        j=j+1
        #print(j)
    print("Filtered down to length:", len(cat_sci_basic))
    cat_sci_basic.write(catalog,overwrite=True,format='ascii.no_header') # hope the headers get written?

In [ ]:
# We wrote a routine to filter the catalogs before normalization to remove flagged sources (generally saturated or
# other bad things) but that did not seem to improve the situation here (sse), so we are giving up! Change this flag
#if you like, but no evidence that it actually helps!
#filter_cat_flag=1
# in theoutlier elimination, also - reduced the spread over the normalization, but didn't actually improve
# the resulting images! arg! This gives a 3 sigma filter on the flux ratios.
#eliminate_outliers_flag=1
#show_images_flag=1
sub_array=[]
#for i in range(len(bkg_subtract_array)-1): 
for i in range(len(sci_conv_array)): # changed this line!
    
    # on dpj some of the galaxy core was left after subtraction, so going to add the saturation
    # keyword to the source extractor, so maybe it will skip some bright sources and the normalization
    # will be better....nope, this did not work!
    sci_image = fits.open(sci_conv_array[i])
    SAT=sci_image[0].header['SATURATE']
    print("Saturation level on Science:",SAT)
    # Run SExtractor on the science image
    if os.path.exists('sci_match.cat'): #Remove possible temporary files - don't think I actually need this!
        os.remove("sci_match.cat") 
    # watch the order on these parameters, apparently putting saturation at the end dooesn't work!
    # NOte these are explicitly being done as ASCII!
    sextractor_command = "source-extractor %s -c prepsfex.sex -CATALOG_NAME sci_match.cat -MAG_ZEROPOINT 25.0 -SATUR_LEVEL %s -DETECT_THRESH %s -CATALOG_TYPE=ASCII_HEAD" % (sci_conv_array[i],SAT,extract_thresh)
    #sextractor_command = "source-extractor %s -c prepsfex.sex -CATALOG_NAME sci_match.cat -MAG_ZEROPOINT 25.0 -CATALOG_TYPE=ASCII_HEAD" % (sci_conv_array[i])
    
    try:
        command = sextractor_command
        print('Executing command: %s\n' % command)
        #command = "source-extractor %s -c %s -CATALOG_NAME %s -MAG_ZEROPOINT 25.0 -SATUR_LEVEL %s" % (sci_conv_array[i], \
        #os.path.join(data_dir,'prepsfex.sex'), os.path.join(proc_dir, 'sci_match.cat'),SAT)
        rval = subprocess.run(command.split(), check=True)
        print('Success!')
    except subprocess.CalledProcessError as err:
        print('Could not run SExtractor with exit error %s'%err)

    if (filter_cat_flag==1): clean_cat("sci_match.cat")
    cat_sci = ascii.read('sci_match.cat') # dummy, should be able to rewrite constantly
    
    ref_image = fits.open(ref_conv_array[i])
    SAT=ref_image[0].header['SATURATE']
    print("Saturation level on Reference",SAT)
    # Run SExtractor on the reference image
    sextractor_command = "source-extractor %s -c prepsfex.sex -CATALOG_NAME ref_match.cat -MAG_ZEROPOINT 25.0 -SATUR_LEVEL %s -DETECT_THRESH %s -CATALOG_TYPE=ASCII_HEAD" % (ref_conv_array[i],SAT,extract_thresh)
    #sextractor_command = "source-extractor %s -c prepsfex.sex -CATALOG_NAME ref_match.cat -MAG_ZEROPOINT 25.0 -CATALOG_TYPE=ASCII_HEAD" % (ref_conv_array[i])
    
    try:
        command = sextractor_command
        print('Executing command: %s\n' % command)
        rval = subprocess.run(command.split(), check=True)
        print('Success!')
    except subprocess.CalledProcessError as err:
        print('Could not run SExtractor with exit error %s'%err)

    # Read in the SExtractor output catalog
    if (filter_cat_flag==1): clean_cat("ref_match.cat")
    cat_ref = ascii.read('ref_match.cat')
    
    # Match the catalog of sources of the reference and science images.  
    # Calculate the ratio between the flux of source in the science image over the flux of sources 
    # in the reference image.
    
    c_sci = SkyCoord(ra=cat_sci['X_WORLD'], dec=cat_sci['Y_WORLD'])
    c_ref = SkyCoord(ra=cat_ref['X_WORLD'], dec=cat_ref['Y_WORLD'])

    # I think idx is the index of c_ref that each c_sci matches into.
    idx, d2d, d3d = c_sci.match_to_catalog_3d(c_ref)

    # Initialize a list for the indexes and one for the flux ratios
    # for each (sci, ref) pair, don't need to save all this?
    index_arr = []
    ratio_arr = []

    # for debugging
    sci_list=[]
    ref_list=[]
    
    # Sure don't need to be printing all this stuff! Is there a final normalization i can use for checking?
    for j, i2, d in zip(idx, np.arange(len(d2d)),d2d):
        #print(i,d)
        #index_arr.append(i)
        
        # for debugging #
        #print("Image coordinates | separation")
        #print(cat_ref['X_IMAGE'][j],cat_ref['Y_IMAGE'][j],'  ', cat_sci['X_IMAGE'][i2],cat_sci['Y_IMAGE'][i2],d)
        #print('Fluxes and flux ratio')
        #print(cat_ref['FLUX_AUTO'][j], cat_sci['FLUX_AUTO'][i2],'                  ', cat_sci['FLUX_AUTO'][i2] / cat_ref['FLUX_AUTO'][i])
        ##
        
        # For debugging
        sci_list.append(cat_sci['FLUX_AUTO'][i2])
        ref_list.append(cat_ref['FLUX_AUTO'][j])
        
        ratio_arr.append(cat_sci['FLUX_AUTO'][i2] / cat_ref['FLUX_AUTO'][j]) # Ratio of SCI/REF!
    
    if (show_images_flag==1):
        plt.xlabel('Source Number')
        plt.ylabel('Flux')
        plt.plot(sci_list)
        plt.plot(ref_list)
        plt.show()
    
    #
    # Current thought is the issue is here - the median vs mean of the scaling factors are vastly
    # different (factor of 4) as opposed to the case for dpj, when they are like 0.9-1.1. 
    # Maybe going back and carrying around a correct value of the saturate keyword will help?
    # another alternative is to go back to the background of the reference image - it's just worse!
    #
    
    
    if (eliminate_outliers_flag==1):
        fig, axs = plt.subplots(1, 2,figsize=(8,8))
        im0=axs[0].set_title('Pre-cleaned flux ratios')
        im0=axs[0].plot(ratio_arr)
        remove = np.where(ratio_arr>np.average(ratio_arr)+3*np.std(ratio_arr))
        print("removing...",remove)
        ratio_arr = np.delete(ratio_arr, remove)
        #for i in reversed(range(len(ratio_arr))): # this should go backward...
        #    print(ratio_arr[i],np.average(ratio_arr),np.std(ratio_arr))
        #    if ( ratio_arr[i]>np.average(ratio_arr)+3*np.std(ratio_arr) ) or ( ratio_arr[i]<np.average(ratio_arr)-3*np.std(ratio_arr) ):
        #        print("eliminating ratio:", ratio_arr[i])
        #        ratio_arr[i].remove()
        im1=axs[1].set_title('Cleaned flux ratios')
        im1=axs[1].plot(ratio_arr)
        plt.show()
    
    scale = np.median(ratio_arr)
    #scale=np.average(ratio_arr) # changing to average didn't seem to help
    print("The scaling factor is", scale,"average:",np.average(ratio_arr)," deivation",np.std(ratio_arr))
    
    # plotting for evaluation
    #plt.hist(ratio_arr)
    #plt.show()
    
    # fine tune the alignment here - at least without this, the alginment is terrible!
    #sci_image = fits.open(sci_conv_array[i])
    #ref_image = fits.open(ref_conv_array[i])
    
    # For testing alignment on koy
    #xoff, yoff, exoff, eyoff = chi2_shift(ref_image[0].data, sci_image[0].data, 10, return_error=True, upsample_factor='auto')
    # errors are just flat zeros here, so not likely they are being calculated carefully.
    # no, those zeros are found in other sources as well...
    xoff, yoff, exoff, eyoff = chi2_shift(ref_image[0].data, sci_image[0].data, 10, return_error=True, upsample_factor=10)
    print("Alignment offsets:",xoff,yoff,exoff,eyoff)
    sci_image_shift = scipy.ndimage.shift(sci_image[0].data, [-yoff, -xoff], order=3, mode='reflect', cval=0.0, prefilter=True)
 
    # Create an empty image with the reference image in a square in the center.
    # "synthetic image"
    #RA=128.249
    #Dec=-27.449311
    coord = SkyCoord(RA,Dec,unit='deg')
    w = WCS(sci_image[0].header)
    x, y = w.world_to_pixel(coord)
    print("The source is at at: ",x,y)

    # The synthetic image will be an image that starts with zeros everywhere, but with the windowed reference image
    # at the center.
    synthetic_image=np.zeros(sci_image[0].data.shape) 
    synthetic_image[int(y)-window:int(y)+window,int(x)-window:int(x)+window]=ref_image[0].data[int(y)-window:int(y)+window,int(x)-window:int(x)+window]
    
    # Do we need to plot the science and reference image here, as part of the debug?
    if (show_images_flag==1):
        fig, axs = plt.subplots(1, 2,figsize=(16,16))
        im0=axs[0].set_title('Science Image')
        im0=axs[0].imshow(sci_image_shift)
        fig.colorbar(im0,ax=axs[0],shrink=0.25)
        im1=axs[1].set_title('Reference (NOT Sythnetic Image)')
        im1=axs[1].imshow(ref_image[0].data)
        fig.colorbar(im1,ax=axs[1],shrink=0.25)
        plt.show()
    
    # image subtraction step
    # Rescale the reference image to the science and perform the image subtraction.
    image_sub=sci_image_shift-synthetic_image*scale
    
    #plot step for evaluation
    # JEsus christ, we have to get the titles in before the imshow command or the colorbar doens't know what to do!
    fig, axs = plt.subplots(1, 3,figsize=(16,16))
    im0=axs[0].set_title('Science Image Subtraction Region')
    im0=axs[0].imshow(sci_image_shift[int(y)-window:int(y)+window,int(x)-window:int(x)+window])
    fig.colorbar(im0,ax=axs[0],shrink=0.25)
    #plt.show()
    
    im1=axs[1].set_title('(scaled) Synthetic Image Subtraction Region')
    im1=axs[1].imshow(scale*synthetic_image[int(y)-window:int(y)+window,int(x)-window:int(x)+window])
    fig.colorbar(im1,ax=axs[1],shrink=0.25)
    #plt.show()
    
    im2=axs[2].set_title('Subtracted Image Subtraction Region')
    #x,y=w.world_to_pixel(coord)
    
    im2=axs[2].imshow(image_sub[int(y)-window:int(y)+window,int(x)-window:int(x)+window])
    im2=axs[2].scatter(window, window, marker='o', color='r', facecolors='none',s=1000,linewidths=2)
    #axs[2].colorbar(shrink = 0.4)
    fig.colorbar(im2,ax=axs[2],shrink=0.25)
    plt.show()
    #plt.tight_layout(h_pad=1)
    
    #image_sub=sci_image_shift-synthetic_image*scale
    hdu_image_sub = fits.PrimaryHDU(image_sub,sci_image[0].header) # header might need work!
    sub_array.append(sci_conv_array[i].replace(".fits","_diff.fits")) 
    hdu_image_sub.writeto("out/"+sub_array[i],overwrite=True) # move to the out directory
    


koy: (R) Alignment clearly did not work...fixed with plate solve worksheet! (U) It looks like the reference image did not match the scientific image for all the images. U is a bad choice for filter! Ok, after taking one out, they at least all look like they are pointed at the right thing. Let's try photometry!

sse: Clearly didn't work - subtracted images look insane. I tried many, many, many things....but actually, I think they are fine! It's just that the saturated stars in the window never get subtracted right (and we should not expect them to!), so they are just always wrong. The background still looks around zero, and the non-saturated sources actually look ok. I think the saturation flag filter is GOOD in these cases, but I think we should move right on to (attempting...) photometry.

umf: Same problem as sse, there are saturated stars in the subtraction region, making the subtractions look wrong. But, I think they are right, and if we want them to look better we've have to totally avoid the saturated stars in the reference, which would make the cutout like 20 x 20. One could run it that way one time to check, but the images actually kinda look fine, so let's try straight on with photometry.

yfh: without saturation mask, and there are still saturated stars in these images. We might want to shrink the sizes of the windows, but let's try to mask the saturated stars first. Yeah that masking actually SCREWED UP the subtraction MORE...go back and change the windows I suppose.

Let's plot them all to get a good look. And also to get an initial flux estimate.

In [ ]:
#fig, axs = plt.subplots(len(sci_conv_array), 2, figsize=(50,50)) # ugh figsize does nothing.
for i in range(len(sci_conv_array)):
    fig, axs = plt.subplots(1, 2, figsize=(50,50))
    sci_image = fits.open(sci_conv_array[i])
    sub_image = fits.open("out/"+sub_array[i])
    
    w = WCS(sub_image[0].header)
    #print(w)
    coord = SkyCoord(ra=RA * u.deg, dec=Dec * u.deg, frame='icrs')
    #print(coord)
    x,y=w.world_to_pixel(coord)
    #print(x,y)
    
    aper = CircularAperture((x,y), 8)
    
    mean, median, std = sigma_clipped_stats(sub_image[0].data)
    axs[0].plot(aper)
    #axs[0].scatter(x, y, marker='o', color='r', facecolors='none',s=1000,linewidths=2)
    axs[0].imshow(sub_image[0].data, vmin=median-2*std, vmax=median+2*std, cmap='gray')
        
    mean, median, std = sigma_clipped_stats(sci_image[0].data)
    axs[1].imshow(sci_image[0].data, vmin=median-2*std, vmax=median+2*std, cmap='gray')
    plt.show()

Evaluating these images is kind of a nightmare, after the PSFing. We need to figure out a way to reduce the number of saturated stars...online calculator? Or is that my job...?

This sheet actually might do that, it looks observatory independent?

https://roman.gsfc.nasa.gov/science/ETC2/ExposureTimeCalc.html